The first step is to create an AI that can play the mineral collection game.

In [1]:
import numpy as np
from pysc2.agents import base_agent
from pysc2.lib import actions
from pysc2.lib import features
from pysc2.env import sc2_env, run_loop, available_actions_printer
from pysc2 import maps
from absl import flags

In [2]:
_AI_RELATIVE = features.SCREEN_FEATURES.player_relative.index
_AI_SELF = 1
_AI_NEUTRAL = 3
_NO_OP = actions.FUNCTIONS.no_op.id
_MOVE_SCREEN = actions.FUNCTIONS.Attack_screen.id
_SELECT_ARMY = actions.FUNCTIONS.select_army.id
_SELECT_ALL = [0]
_NOT_QUEUED = [0]

In [5]:
def get_beacon_location(ai_relative_view):
    '''returns the location indices of the beacon on the map'''
    return (ai_relative_view == _AI_NEUTRAL).nonzero() 
    
class Agent2(base_agent.BaseAgent):
    """An agent for doing a simple movement form one point to another."""
    def step(self, obs):
        '''step function gets called automatically by pysc2 environment'''
        super(Agent2, self).__init__()
        super(Agent2, self).step(obs)
        if _MOVE_SCREEN in obs.observation['available_actions']:
            ai_view = obs.observation['screen'][_AI_RELATIVE]
            # get the beacon coordinates
            beacon_xs, beacon_ys = get_beacon_location(ai_view)
            if not beacon_ys.any():
                return actions.FunctionCall(_NO_OP, [])
            # get the middle of the beacon and move there
            target = [beacon_ys.mean(), beacon_xs.mean()]
            return actions.FunctionCall(_MOVE_SCREEN, [_NOT_QUEUED, target])
        else:
            return actions.FunctionCall(_SELECT_ARMY, [_SELECT_ALL])

In [6]:
FLAGS = flags.FLAGS
FLAGS(['run_sc2'])

viz = True
steps = 25000
save_replay = True

# create a map
beacon_map = maps.get('MoveToBeacon')

# create an envirnoment
with sc2_env.SC2Env(agent_race=None, bot_race=None, difficulty=None, map_name=beacon_map, visualize=viz) as env:
    agent=Agent2()
    run_loop.run_loop([agent], env, steps)
    if save_replay:
        env.save_replay(Agent2.__name__)

Took 7.970 seconds for 2102 steps: 263.755 fps


Reward:

In [ ]:
agent.reward, agent.episodes

In [ ]:
agent.reward/agent.episodes

Much better than 0! This is an avg reward close to 18 per episode. 

Let's say that a reward of 18-20 is optimal per episode. Now we have some idea for what to benchmark our algorithm to.

In [ ]:
env.step